# PyTorch-VGG-ONNX-Tensorflow

## Use PyTorch pretrained model

In [1]:
import torch
import torchvision.models as models
import os

# Use an existing model from Torchvision, note it 
# will download this if not already on your computer (might take time)
net = models.vgg16(pretrained=True)
net.eval()

# Create some sample input in the shape this model expects
dummy_input = torch.ones(10, 3, 224, 224)

In [2]:
import os
if not os.path.isdir("model"):
    os.mkdir("model")

model_name = "pytorch_vgg.onnx"
model_path = os.path.join("model", model_name)
torch.onnx.export(net, 
                  dummy_input, 
                  model_path, 
                  verbose=True,
                  export_params=True
                 )

graph(%0 : Float(10, 3, 224, 224)
      %1 : Float(64, 3, 3, 3)
      %2 : Float(64)
      %3 : Float(64, 64, 3, 3)
      %4 : Float(64)
      %5 : Float(128, 64, 3, 3)
      %6 : Float(128)
      %7 : Float(128, 128, 3, 3)
      %8 : Float(128)
      %9 : Float(256, 128, 3, 3)
      %10 : Float(256)
      %11 : Float(256, 256, 3, 3)
      %12 : Float(256)
      %13 : Float(256, 256, 3, 3)
      %14 : Float(256)
      %15 : Float(512, 256, 3, 3)
      %16 : Float(512)
      %17 : Float(512, 512, 3, 3)
      %18 : Float(512)
      %19 : Float(512, 512, 3, 3)
      %20 : Float(512)
      %21 : Float(512, 512, 3, 3)
      %22 : Float(512)
      %23 : Float(512, 512, 3, 3)
      %24 : Float(512)
      %25 : Float(512, 512, 3, 3)
      %26 : Float(512)
      %27 : Float(4096, 25088)
      %28 : Float(4096)
      %29 : Float(4096, 4096)
      %30 : Float(4096)
      %31 : Float(1000, 4096)
      %32 : Float(1000)) {
  %33 : Float(10, 64, 224, 224) = onnx::Conv[dilations=[1, 1], group=1, kern

## Import the ONNX model to Tensorflow

In [3]:
import onnx
from onnx_tf.backend import prepare

# Load the ONNX file
model = onnx.load("model/pytorch_vgg.onnx")

# Import the ONNX model to Tensorflow
tf_rep = prepare(model)

/usr/local/lib/python3.5/dist-packages/onnx_tf/common/__init__.py:87: UserWarning: FrontendHandler.get_outputs_names is deprecated. It will be removed in future release.. Use node.outputs instead.
  warnings.warn(message)
/usr/local/lib/python3.5/dist-packages/onnx_tf/common/handler_helper.py:73: UserWarning: Unknown op Asinh in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/usr/local/lib/python3.5/dist-packages/onnx_tf/common/handler_helper.py:73: UserWarning: Unknown op Cosh in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/usr/local/lib/python3.5/dist-packages/onnx_tf/common/handler_helper.py:73: UserWarning: Unknown op Sinh in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/usr/local/lib/python3.5/dist-packages/onnx_tf/common/handler_helper.py:73: UserWarning: Unknown op Atanh in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/usr/local/lib/python3.5/dist-packages/onnx_tf/common/handler_helper.py:73: 

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


In [4]:
import numpy as np

tf_out = tf_rep.run(dummy_input.numpy())
torch_out = net(dummy_input).detach().numpy()

np.testing.assert_almost_equal(tf_out[0], 
                               torch_out, 
                               decimal=3
                              )

print("Exported model has been executed on TF backend, and the result looks good!")

Exported model has been executed on TF backend, and the result looks good!
